In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd

In [3]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [4]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [5]:
#Create an engine to connect to the database
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
conn = engine.connect()

In [6]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

In [7]:
# We can view all of the classes that automap found
Base.classes.keys()


['measurement', 'station']

In [8]:
# Save references to each table

# Assign the measurement class to a variable called "Measurement"
Measurement = Base.classes.measurement

# Assign the station class to a variabe called "Station"
Station = Base.classes.station

In [9]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [10]:
# Display the row's columns and data in dictionary format
first_row = session.query(Measurement).first()
first_row.__dict__


{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1c6c8c3e708>,
 'date': '2010-01-01',
 'tobs': 65.0,
 'id': 1,
 'prcp': 0.08,
 'station': 'USC00519397'}

In [11]:
# Display the row's columns and data in dictionary format
first_row = session.query(Station).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1c6c8c20808>,
 'station': 'USC00519397',
 'latitude': 21.2716,
 'elevation': 3.0,
 'longitude': -157.8168,
 'id': 1,
 'name': 'WAIKIKI 717.2, HI US'}

# Exploratory Climate Analysis

In [13]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results
precip = engine.execute("SELECT * FROM measurement")

for record in precip:
        print (record)

# Calculate the date 1 year ago from the last data point in the database

# Perform a query to retrieve the data and precipitation scores

# Save the query results as a Pandas DataFrame and set the index to the date column

# Sort the dataframe by date

# Use Pandas Plotting with Matplotlib to plot the data


.38, 70.0)
(19140, 'USC00516128', '2016-06-16', 0.12, 70.0)
(19141, 'USC00516128', '2016-06-17', 1.27, 69.0)
(19142, 'USC00516128', '2016-06-18', 0.72, 73.0)
(19143, 'USC00516128', '2016-06-19', 0.79, 74.0)
(19144, 'USC00516128', '2016-06-20', 0.52, 72.0)
(19145, 'USC00516128', '2016-06-21', 0.78, 71.0)
(19146, 'USC00516128', '2016-06-22', 0.2, 73.0)
(19147, 'USC00516128', '2016-06-23', 0.9, 72.0)
(19148, 'USC00516128', '2016-06-25', None, 73.0)
(19149, 'USC00516128', '2016-06-26', 0.7, 74.0)
(19150, 'USC00516128', '2016-06-27', 0.3, 72.0)
(19151, 'USC00516128', '2016-06-28', 0.25, 74.0)
(19152, 'USC00516128', '2016-06-29', 0.35, 72.0)
(19153, 'USC00516128', '2016-07-01', None, 74.0)
(19154, 'USC00516128', '2016-07-04', None, 74.0)
(19155, 'USC00516128', '2016-07-05', 0.45, 71.0)
(19156, 'USC00516128', '2016-07-06', 1.46, 71.0)
(19157, 'USC00516128', '2016-07-07', 0.92, 73.0)
(19158, 'USC00516128', '2016-07-08', 0.31, 73.0)
(19159, 'USC00516128', '2016-07-09', 0.15, 71.0)
(19160, 'USC0

In [53]:
# Use Pandas to calcualte the summary statistics for the precipitation data

In [54]:
# Design a query to show how many stations are available in this dataset?


In [55]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.


In [56]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?


In [57]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


## Bonus Challenge Assignment

In [58]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

NameError: name 'session' is not defined

In [59]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [60]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [61]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



In [62]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

NameError: name 'Measurement' is not defined

In [63]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [64]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [65]:
# Plot the daily normals as an area plot with `stacked=False`
